# Week3_4 Assignment

## [BASIC](#Basic) 
- Encoder & Decoder Layer 코드를 직접 필사하고 각 함수에 주석을 달 수 있다. 

## [CHALLENGE](#Challenge)
- 텐서의 크기(shape)를 계산할 수 있다. 

## [ADVANCED](#Advanced)
- 완성된 transformer 모델의 모든 학습 가능한 파라미터 이름과 크기(shape)를 출력할 수 있다.

### Informs
논문 정리 블로그 : [Transformer 논문 제대로 읽기](https://velog.io/@crosstar1228/NLP-Attention-is-all-you-need-Transformer-%EB%85%BC%EB%AC%B8-%EC%A0%9C%EB%8C%80%EB%A1%9C-%EC%9D%BD%EA%B8%B0)
이번 과제에서는 "[Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)"의 코드를 필사해본다.   
"Annotated Transformer"는 "Attention is all you need" 논문에서 제안한 transformer 모델을 pytorch 라이브러리로 직접 구현한다.   
코드 필사를 통해 다음을 배울 수 있다.    
- Encoder, Decoder 구조
- Attention Mechanism
- "residual connection", "layer normalization" 등의 구조 

코드 필사를 시작하기 앞서, transformer 모델의 최종 구조를 살펴보자.    

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_full.png?raw=true" width="500" align="center"/>

최종 모델은 `EncoderDecoder()` 클래스에 여러 인스턴스를 생성자의 입력 파라미터로 넣어 생성한다.    
앞으로 우리는 `EncoderDecoder()` 클래스와 같은 여러 클래스들을 구현하고 연결할 것이다. 따라서 대략적인 클래스간의 관계를 살펴보고 이해한다면 보다 큰 그림을 가지고 코드 필사를 할 수 있을 것이다. 

Transformer 모델은 크게 4가지 클래스로 구현된다.    
- Frame
    - frame 역할을 하는 `EncoderDecoder` 클래스
- Input Embedding & Encoding
    - 입력값을 벡터화하는 `Embeddings`, `PositionalEncoding`
- Encoder & Decoder
    - 각 6개 layer를 갖고 있는 `Encoder`, `Decoder`
    - layer 1층을 구현한 `EncoderLayer`, `DecoderLayer`
- Sublayer
    - `EncoderLayer`, `DecoderLayer` 내부에서 사용되는 Sublayer 클래스인 `MultiHeadAttiontion`, `PositionwiseFeedForward`
    - Sublayer 클래스들을 연결하는 `SublayerConnection`
    
아래 좌측 도식에서 각 클래스의 색상은 아래 우측 도식(transformer 구조)의 색상과 맵핑되어 있다.    
각 클래스의 역할과 클래스 간 연결 관계를 생각하면서 transformer를 코드로 구현해보자.   


<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_map.png?raw=true" width="400" height="400" align="left"/>
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_transformer.png?raw=true" width="300" height="400" align="right"/>



In [1]:
import os
import sys
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

## Basic

### Frame
- `EncoderDecoder`

아래 도식은 `EncoderDecoder` 클래스의 `forward()`, `encode()`, `decode()` 메소드를 도식화 한 것이다.    
 
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_encoderdecoder.png?raw=true" width=500>


- `Generator`

In [2]:

class EncoderDecoder(nn.Module):
    '''
     transformer encoder-deocoder 구조의 뼈대입니다.
    '''
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
      super(EncoderDecoder, self).__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.src_embed = src_embed # source를 input으로 받아 embedding 으로 바꿔줌
      self.tgt_embed = tgt_embed # target를 input으로 받아 embedding 으로 바꿔줌
      self.generator = generator 
    
    # encoding 및 decoding 전 과정 forward propagation
    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask) 
    
    # encoder에 source 와 masking 정보를 input으로
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)  
    
    # decoder에 encoder output, source mask 및 target embedding과 target masking 정보 넣음 
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask) 


In [3]:
class Generator(nn.Module):
    
    def __init__(self, d_model, vocab):
      super(Generator, self).__init__()
      self.proj = nn.Linear(d_model, vocab) # d_model 차원의 모델의 embedding vector를 통해 vocab 차원의 output값 생성
    
    # vocab 개수만큼의 output 값들을 모아 softmax로 확률 계산
    def forward(self, x):
      return F.log_softmax(self.proj(x), dim=-1) 

### Encoder
- `Encoder`
- `EncoderLayer`
- `SublayerConnection`
- Reference
    - Layer Normalization
        - [한국어 설명](https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/)
        - [torch official docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
    - Residual Connection
        - [한국어 설명](https://itrepo.tistory.com/36)
    - pytorch ModuleList
        - [torch official docs](https://pytorch.org/docs/1.9.1/generated/torch.nn.ModuleList.html)


In [4]:
# N개의 module 생성. default 값 6 (class 아닌 함수)
def clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [5]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
      super(Encoder,self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm(layer.size) # layer size는 곧 feature의 개수
    
    # layer별로 input과 masking 정보를 통과시킴
    def forward(self, x, mask):
      for layer in self.layers:
        x = layer(x, mask) 
      return self.norm(x)

In [6]:
# 데이터 별 feature들을 normalization 해주는 함수 

class LayerNorm(nn.Module):
    
    # feature들이 list를 
    def __init__(self, features, eps=1e-6):
      super(LayerNorm,self).__init__()
      self.a_2 = nn.Parameter(torch.ones(features))
      self.b_2 = nn.Parameter(torch.zeros(features))
      self.eps = eps

    def forward(self, x):
      # 마지막 차원에 feature들에 해당하는 값
      mean = x.mean(-1, keepdim=True)
      std = x.std(-1, keepdim=True)
      return self.a_2 * (x - mean) / (std + self.eps) + self.b_2 # torch.ones 에 standardization 된 값들을 곱하고, 영벡터 더함



In [7]:
class SublayerConnection(nn.Module):
    
    def __init__(self, size, dropout):
      super(SublayerConnection, self).__init__()
      self.norm = LayerNorm(size)
      # 각 sub-layer의 output에 dropout 적용(Residual computation, 그러니까 input과 더해지고 layer normalize 되는 연산을 진행하기 전에).
      self.dropout = nn.Dropout(dropout)
    #  layernorm( x + F(x))
    def forward(self, x, sublayer):
      return x + self.dropout(sublayer(self.norm(x)))

In [8]:
# attention , ffw layer + connection
class EncoderLayer(nn.Module):
    
    def __init__(self, size, self_attn, feed_forward, dropout):
      super(EncoderLayer, self).__init__()
      self.self_attn = self_attn
      self.feed_forward = feed_forward
      self.sublayer = clones(SublayerConnection(size, dropout), 2) # size 만큼의 layer normalization
      self.size = size
        
    def forward(self, x, mask):
      x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)) # self attention -> query key value 모두 자체적인 output
      return self.sublayer[1](x, self.feed_forward) # masking

### Decoder
- `Decoder`
- `DecoderLayer`

In [18]:
# 6개이고 각 layer 별로 3개의 sub layer
# masked multihead self attention + multihead attention of encoder output+ position-wise FC FFN
class Decoder(nn.Module):
    
    def __init__(self, layer, N):
      super(Decoder, self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm(layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
      for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
      return self.norm(x)

In [10]:
class DecoderLayer(nn.Module):

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
      super(DecoderLayer, self).__init__()
      self.size = size
      self.self_attn = self_attn
      self.src_attn = src_attn
      self.feed_forward = feed_forward
      self.sublayer = clones(SublayerConnection(size, dropout), 3)
    
    
    def forward(self, x, memory, src_mask, tgt_mask):
      m = memory
      # query, key, value
      x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask)) 
      # query가 이전의 decoder layer로부터 오고, key와 value는 encoder의 output으로부터 옴
      # query 는 x      
      # m, m 은 각각 encoder output의 key, value
      x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
      return self.sublayer[2](x, self.feed_forward)

### Sublayer
- `attention` 함수

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_attention.png?raw=true" width="500" align="center"/>  

- `MultiHeadedAttention`
- `PositionwiseFeedForward`

### Challenge


### Q1. 위 도식에 따라 `score`, `p_attn`, `attention` 을 구하라 
$$attention(Q,K,V) = softmax( \frac {QK^T} {\sqrt{d_k}}) V$$

In [11]:
def attention(query, key, value, mask=None, dropout=None):
  d_k = query.size(-1)
  # matrix multiplication( -2, -1 차원을 transpose)
  scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
  # 0으로 masking
  if mask is not None: 
  # 해당 지점 0으로 마스킹
        scores = scores.masked_fill(mask == 0, -1e9)
  p_attn = F.softmax(scores, dim = -1)
  # dropout layer 통과
  if dropout is not None:
      p_attn = dropout(p_attn)
  return torch.matmul(p_attn, value), p_attn


###Q2. query, key, value가 모두 (m, d_k) shape의 matrix라고 가정할 때, `score`, `p_attn`, `attention`의 shape을 각각 구하라

- `m` : input sequence token 개수
- `d_k` : embedding vector의 차원/head
---
- score : (`m` , `m`)
- p_attn : (`m`, `m`)
- attention : (`m`, `d_k`)


### (아래의 **Q3을 먼저 풀고 돌아오세요**) Q4.  query, key, value가 모두 (12, 8, 1, 64) shape의 tensor라고 가정할 때 , `score`, `p_attn`, `attention`의 shape을 각각 구하라

- score : (12,8,1,1)
- p_attn : (12,8,1,1)
- attention : (12,8,1,64)

- `MultiHeadedAttention`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_multihead.png?raw=true" width="300" align="center"/>  

![](https://images.velog.io/images/crosstar1228/post/3db389a4-a792-4c58-a400-fb97ac3ec89d/image.png)
- $$W_i^Q, W_i^K, W_i^V$$ 는 각각 query, key, value의 projection 된 parameter matrix
- $$W_i^Q \in R^{d_{model} * d_k}$$, $$W_i^K \in R^{d_{model} * d_k}$$, $$W_i^V \in R^{d_{model} * d_v}$$
- $$W^o \in R^{{h d_v}* d_{model}}$$ : head 개수 만큼 concat 했으므로

In [12]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
      super(MultiHeadedAttention, self).__init__()
      assert d_model % h == 0
      # d_v == d_k 가정
      self.d_k = d_model // h
      self.h = h
      self.linears = clones(nn.Linear(d_model, d_model), 4)
      self.attn = None
      self.dropout = nn.Dropout(p=dropout)
    
    
    def forward(self, query, key, value, mask=None):
      if mask is not None:
          # 같은 mask가 모든 h값에 매핑
          mask = mask.unsqueeze(1) 
      nbatches = query.size(0) # 0번째 축은 batch size

      # 1) (n_batches, h, 1, d_k) 모양으로 project 
      query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
                              for l, x in zip(self.linears, (query, key, value))]
        
      # 2) attention 연산 수행
      x, self.attn = attention(query, key, value, mask=mask,dropout=self.dropout)
        
      # 3) view 이용하여 concat 
      x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
      return self.linears[-1](x)

### Q3.  query, key, value가 모두 (12, 512) shape의 matrix이고, h 값이 8 이라고 가정할 때, 아래 값의 shape을 각각 구하라

- `d_k` (d_k = d_model // h) : 512/8  = 64
- `nn.Linear(d_model, d_model)(query)` : (12,512(d_model))
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k)` : (12, 1, 8, 64)
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2)` : (12, 8, 1, 64) 

- `PositionwiseFeedForward`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pwff.png?raw=true" width="300" align="center"/>  

In [13]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff) # input size d_model
        self.w_2 = nn.Linear(d_ff, d_model) # hidden size d_ff
        self.dropout = nn.Dropout(dropout)

  def forward(self, x):
      return self.w_2(self.dropout(F.relu(self.w_1(x)))) #w_1 -> relu -> dropout -> w_2

### Input Embedding & Encoding
- `Embeddings`
    - [pytorch official docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [14]:
class Embeddings(nn.Module):
     # vocab, d_model 의 모양의 lookup table 생성
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model
    # lookup table * sqrt(d_model)
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

- `PositionalEncoding`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pe.png?raw=true" width="500" align="center"/>  

- `position` 변수 설명
    - 모든 position (=최대 토큰 개수)의 값을 갖고 있는 matrix
- `div_term` 변수 설명

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_div.png?raw=true" width="500" align="center"/>  
- `Embedding` + `Encoding` 도식화 

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_emb_enc.png?raw=true" width="400" align="center"/>  


In [15]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # positional encoding 연산
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    # gradient 계산이 가능하게 만들어 준 후 input embedding 값에 더해줌    
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

### Q4.  max_len이 512이고, d_model이 512라고 가정할 때, `position`과 `div_term`의 shape을 구하라

- `position` : (512,1)
- `div_term` : (256,)
- `position * div_term` : (512,256)

### Advanced

### Finally Build Model
- Xavier Initialization
    - [한국어 자료](https://huangdi.tistory.com/8)
    - [pytorch official docs](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_)

In [16]:
def make_model(src_vocab, tgt_vocab, 
               N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
  c = copy.deepcopy
  attn = MultiHeadedAttention(h, d_model)  # d_k 차원의 h개의 head로 나누어 attention 연산 수행
  ff = PositionwiseFeedForward(d_model, d_ff, dropout) # d_model 차원의 데이터를 input으로 받아 hiddensize가 d_ff 인 연산 수행
  position = PositionalEncoding(d_model, dropout) #  d_model 차원의 positional encoding 수행
  model = EncoderDecoder(
      Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N), #(self attention + ff) 의 sub layer
      Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                            c(ff), dropout), N), #(selfattention +enc_dec attention+ ff) 의 sub layer
      nn.Sequential(Embeddings(d_model, src_vocab), c(position)), #source의 embedding vector + position
      nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)), # target의 embedding vector + position
      Generator(d_model, tgt_vocab)) # output embedding으로부터 softmax 연산 진행하여 최종 예측값 반환
  
  # This was important from their code. 
  # Initialize parameters with Glorot / fan_avg.
  for p in model.parameters():
      if p.dim() > 1:
          nn.init.xavier_uniform(p)
  return model
            

In [19]:
model = make_model(10,10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


### Q5. 위 코드로 만든 모델의 모든 파라미터의 이름과 크기 (shape) 을 출력하라

In [23]:
# 구현

for i, (name, param) in enumerate(model.named_parameters()):
  print(f'number {i} Layer name is : {name},\n and its shape of parameter vector is {param.shape}')
  print('--------------------------------------------------\n')

number 0 Layer name is : encoder.layers.0.self_attn.linears.0.weight,
 and its shape of parameter vector is torch.Size([512, 512])
--------------------------------------------------

number 1 Layer name is : encoder.layers.0.self_attn.linears.0.bias,
 and its shape of parameter vector is torch.Size([512])
--------------------------------------------------

number 2 Layer name is : encoder.layers.0.self_attn.linears.1.weight,
 and its shape of parameter vector is torch.Size([512, 512])
--------------------------------------------------

number 3 Layer name is : encoder.layers.0.self_attn.linears.1.bias,
 and its shape of parameter vector is torch.Size([512])
--------------------------------------------------

number 4 Layer name is : encoder.layers.0.self_attn.linears.2.weight,
 and its shape of parameter vector is torch.Size([512, 512])
--------------------------------------------------

number 5 Layer name is : encoder.layers.0.self_attn.linears.2.bias,
 and its shape of parameter vec